In [1]:
import sys 
import json
import pandas as pd

sys.path.append("intial_eval_utils")

from intial_eval_utils.evaluating_over_debiasing_gpt2 import add_predicts, get_vios_and_dif
from intial_eval_utils.gpt2_ori_seat import get_embeddings, compute_effect_size
from intial_eval_utils.evaluate_by_divers_tmps_gpt_2 import convert_tmps_to_sents, comput_asso_for_autoregressive, add_assos
from intial_eval_utils.bias_utils.debiasing_effects_util import get_effect_size

eval_type = 'test'

GPT_2

Before Debiasing

In [3]:
tmp_path = 'data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents('data/simple_patterns', eval_type = eval_type\
                                                 , simple = True)
corpus, sent2emb = comput_asso_for_autoregressive(corpus, tmps_type, temp_size, model_type = 'gpt2')

seat_effect_size, seat_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))

load tmps from ../data/simple_patterns
--- 1136 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2 
****** need_reload :  True
****** Need to reload...


100%|██████████████████████████████████████████████████████████████████████████████| 1136/1136 [00:16<00:00, 70.96it/s]


For SEAT: effect size is 0.7465725130985419, p value is 0.15446


In [4]:
if eval_type == 'val':
    ori_seat_corpus_path = 'data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = 'data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = 'data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'gpt2')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2


  0%|                                                                                          | 0/234 [00:00<?, ?it/s]

****** need_reload :  True
****** Need to reload...


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:03<00:00, 76.17it/s]


For standard SEAT: effect size is 0.28515287289385016, p value is 0.22553


In [5]:
data_without_preds_path = 'data/2610_sents_for_evaluating_gender_loss.tsv'

data_without_preds = pd.read_csv(data_without_preds_path, sep='\t', index_col = 0)

data = add_predicts(data_without_preds, model_type = 'gpt2')
violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

Loading the model and the tokenizer from gpt2


100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [01:11<00:00, 36.63it/s]

On UAL_Gender
{'error_num': 609, 'error_rate': 0.23333333333333334, 'IA': 0.0014952398788587473}


After Sent_debias

In [6]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents('data/simple_patterns',\
                                                 eval_type = eval_type, simple = True)
corpus, sent2emb = comput_asso_for_autoregressive(corpus, tmps_type, temp_size, model_type = 'gpt2'\
                                                 , subspace_path = 'gpt2_saved_embeddings/base_sent/gender_dir_1.0')

seat_effect_size, seat_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))

load tmps from ../data/simple_patterns
* load gender dir from  gpt2_saved_embeddings/base_sent/gender_dir_1.0
--- 1136 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2 


  0%|                                                                                         | 0/1136 [00:00<?, ?it/s]

****** need_reload :  True
****** Need to reload...


100%|██████████████████████████████████████████████████████████████████████████████| 1136/1136 [00:15<00:00, 73.45it/s]


debiasing by sent_debias...
For SEAT: effect size is 0.023375177096949824, p value is 0.96256


In [7]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'gpt2'\
                                                 , subspace_path = 'gpt2_saved_embeddings/base_sent/gender_dir_1.0')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

* load gender dir from  gpt2_saved_embeddings/base_sent/gender_dir_1.0
--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2


  0%|                                                                                          | 0/234 [00:00<?, ?it/s]

****** need_reload :  True
****** Need to reload...


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:03<00:00, 76.18it/s]


debiasing by sent_debias...
For standard SEAT: effect size is -0.026900173228359987, p value is 0.9085


In [2]:
data_without_preds_path = '../data/2610_sents_for_evaluating_gender_loss.tsv'

data_without_preds = pd.read_csv(data_without_preds_path, sep='\t', index_col = 0)

data = add_predicts(data_without_preds, model_type = 'gpt2'\
                                                 , subspace_path = 'gpt2_saved_embeddings/base_sent/gender_dir_1.0')
violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

* load gender dir from  gpt2_saved_embeddings/base_sent/gender_dir_1.0
Loading the model and the tokenizer from gpt2


100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [01:15<00:00, 34.41it/s]


On UAL_Gender
{'error_num': 1934, 'error_rate': 0.7409961685823755, 'IA': -5.773676530009415e-08}


GPT_2_MEDIUM

Before Debiasing

In [10]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents('../data/simple_patterns', eval_type = eval_type,\
                                                 simple = True)
corpus, sent2emb = comput_asso_for_autoregressive(corpus, tmps_type, temp_size, model_type = 'gpt2-medium')

seat_effect_size, seat_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))

load tmps from ../data/simple_patterns
--- 1136 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2-medium 
****** need_reload :  True
****** Need to reload...


100%|██████████████████████████████████████████████████████████████████████████████| 1136/1136 [00:28<00:00, 40.30it/s]


For SEAT: effect size is -0.2981796581707961, p value is 0.57473


In [11]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'gpt2-medium')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2-medium
****** need_reload :  True
****** Need to reload...


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:05<00:00, 45.97it/s]


For standard SEAT: effect size is -0.3297822561549808, p value is 0.15841


In [12]:
data_without_preds_path = '../data/2610_sents_for_evaluating_gender_loss.tsv'

data_without_preds = pd.read_csv(data_without_preds_path, sep='\t', index_col = 0)

data = add_predicts(data_without_preds, model_type = 'gpt2-medium')
violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

Loading the model and the tokenizer from gpt2-medium


100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [02:23<00:00, 18.24it/s]


On UAL_Gender
{'error_num': 413, 'error_rate': 0.1582375478927203, 'IA': 0.002433732236244269}


After Sent_debias

In [13]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents('../data/simple_patterns', eval_type = eval_type,\
                                                 simple = True)
corpus, sent2emb = comput_asso_for_autoregressive(corpus, tmps_type, temp_size, model_type = 'gpt2-medium'\
                                                 , subspace_path = 'gpt2_saved_embeddings/large_sent/gender_dir_1.0')

seat_effect_size, seat_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))

load tmps from ../data/simple_patterns
* load gender dir from  gpt2_saved_embeddings/large_sent/gender_dir_1.0
--- 1136 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2-medium 
****** need_reload :  True
****** Need to reload...


100%|██████████████████████████████████████████████████████████████████████████████| 1136/1136 [00:28<00:00, 39.22it/s]


debiasing by sent_debias...
For SEAT: effect size is 0.011585961206264062, p value is 0.98445


In [14]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'gpt2-medium'\
                                                 , subspace_path = 'gpt2_saved_embeddings/large_sent/gender_dir_1.0')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

* load gender dir from  gpt2_saved_embeddings/large_sent/gender_dir_1.0
--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
Loading the model and the tokenizer from gpt2-medium
****** need_reload :  True
****** Need to reload...


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:05<00:00, 42.77it/s]


debiasing by sent_debias...
For standard SEAT: effect size is -0.07583661983308462, p value is 0.74747


In [3]:
data_without_preds_path = '../data/2610_sents_for_evaluating_gender_loss.tsv'

data_without_preds = pd.read_csv(data_without_preds_path, sep='\t', index_col = 0)

data = add_predicts(data_without_preds, model_type = 'gpt2-medium'\
                                                 , subspace_path = 'gpt2_saved_embeddings/large_sent/gender_dir_1.0')
violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

* load gender dir from  gpt2_saved_embeddings/large_sent/gender_dir_1.0
Loading the model and the tokenizer from gpt2-medium


100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [02:23<00:00, 18.25it/s]


On UAL_Gender
{'error_num': 739, 'error_rate': 0.28314176245210726, 'IA': 9.054299421669588e-08}
